In [101]:
from pybaselines import Baseline, utils
import pandas as pd
import spectral
import pysptools
from spectral import *
from matplotlib import pyplot as plt
import numpy as np
from scipy import signal, interpolate, stats
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
import pickle
from datetime import datetime
import os
from tqdm.notebook import tqdm
from scipy.optimize import nnls
from numpy.linalg import norm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import display
# import glob

In [109]:
# Define some functions for later

def compute_EGO_Gaussian(lmbda, si, mu_i, sigma_i, k_i):
    # EGO Gaussian function
    return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))

def weighted_sum_EGO_Gaussians(x, *params):
    num_peaks = len(params) // 4
    result = np.zeros_like(x)
    for i in range(num_peaks):
        si, mu_i, sigma_i, k_i = params[i*4 : (i+1)*4]
        result += compute_EGO_Gaussian(x, si, mu_i, sigma_i, k_i)
    return result

def find_initial_guess_EGO(x,
                           y,
                           width = 5):
    """
    Uses the initial spectrum to retrieve peaks using SciPy's
    find_peaks function. Returns the data in a format that we 
    can use for the initial guess for optimization, specifically
    for the 4 parameter EGO (exponential gaussian) we're interested
    in. This means it returns each peak with a default width, amplitude,
    and asymmetry, but identifies the center of the distribution 
    
    """
    peaks, _ = find_peaks(y,
                          width=width)
    initial_guess = []
    for p in peaks:
        initial_guess += [1.0, x[p], 10, -0.1]
    return initial_guess

# Function to open, load and close a pickled file
def load_data(file_name):
    with open(file_name, 'rb') as file:
        data = pickle.load(file)
    return data

def do_fit(curr_x,
                curr_y,
                curr_start,
                curr_end,
                curr_spectrum,
                wav_array,
                fit_type = 'EGO'):
    """
    This function is going to be primarily concerned with performing the 
    actual optimization for a specific spectrum, or a subset of a spectrum.
    It is designed to be re-run depending on a variety of conditions, including
    if the fit fails, or if it produces a fit below an R2 threshold
    """
    # Process into 0 to 1 format

    curr_peak = curr_spectrum[curr_start:curr_end]
    curr_peak = curr_peak / curr_peak.max()
    curr_peak = 1 - curr_peak
    # Truncate the wavelengths too
    curr_wavs = wav_array[curr_start:curr_end]

    # Find initial guess for the parameters
    # This is insufficient for cases where find_peaks
    # fails to find more than 0 or 1 peaks
    if fit_type == 'EGO':
        initial_guess_EGO = find_initial_guess_EGO(curr_wavs,
                                                   curr_peak)

        # Fit the weighted sum of EGO Gaussians to the data
        params, _ = curve_fit(weighted_sum_EGO_Gaussians,
                              curr_wavs,
                              curr_peak,
                              p0=initial_guess_EGO)
        
        # Generate y-values for the fitted curve using the obtained parameters
        y_values = weighted_sum_EGO_Gaussians(curr_wavs, *params)

    return params, y_values, curr_wavs, curr_peak

def attempt_fit(curr_x,
                curr_y,
                curr_start,
                curr_end,
                curr_spectrum,
                wav_array,
                fit_type = 'EGO'):
    try:
        params, y_values, curr_wavs, curr_peak = do_fit(curr_x,
                                                       curr_y,
                                                       curr_start,
                                                       curr_end,
                                                       curr_spectrum,
                                                       wav_array,
                                                       fit_type = 'EGO')
        return True, params, y_values,  curr_wavs, curr_peak
    except RuntimeError:
        # print('Optimization failed, attempting to generate a successful fit')
        return False, [], [], [], []
        
    
    

In [26]:
# Load the data
input_hulls = 'baseline_hybrid_corrected_array.pickle'
baseline_corrected_data = load_data(input_hulls)
# Assume that we get the wav_array from somewhere
wav_array = np.round(np.linspace(448,2500,baseline_corrected_data.shape[2]))

In [ ]:
# Store All the resulting data in a dictionary by pixel
peak_data = {}

# Begin a loop to process each pixel 

for curr_x in tqdm(range(baseline_corrected_data.shape[0])):
    pbar = tqdm(range(baseline_corrected_data.shape[1]))
    # for char in pbar:
    for curr_y in pbar :
        # curr_x = 0
        # curr_y = 0 
        # Initialize the truncation parameters
        curr_start = 0
        curr_end = baseline_corrected_data.shape[2]
        # And a variable to identify if we're playing with them 
        truncated_start = True
        truncated_end = False
        fit_success = False
        # Easier if iteration is tracked from -1 to start truncation that way
        curr_iteration = -1
        truncate_range = np.arange(0,200,10)
        
        # Load the current spectrum
        curr_spectrum = baseline_corrected_data[curr_x,curr_y,:]
        pbar.set_description(f"Processing pixel {curr_x}, {curr_y}")
        # Attempt to fit
        # First attempt will just handle truncation to attempt to succeed
        while fit_success is False and truncated_start is True:
            curr_iteration+=1
            if curr_iteration < truncate_range.shape[0]:
                fit_success, params, y_values, curr_wavs, curr_peak = attempt_fit(curr_x,
                                                                                   curr_y,
                                                                                   curr_start=truncate_range[curr_iteration],
                                                                                   curr_end = curr_end,
                                                                                   curr_spectrum = curr_spectrum,
                                                                                   wav_array = wav_array)
            else:
                fit_success = False
                truncated_start = False
                params = []
                y_values = []
                curr_wavs = []
                curr_peak = []

        if fit_success:
            # calculate r_squared and SSE        
            residuals = curr_peak - y_values
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((curr_peak-np.mean(curr_peak))**2)
            r_squared = 1 - (ss_res / ss_tot)
            sse = np.sum((curr_peak - y_values)**2)
            
            # Here we should prepare to re-do the optimization if we don't hit
            # a reasonable R2 threshold
            # We should also consider an optimization step here, 
            # Bayesian hyperparam or grid search, that would 
            # allow us to guarantee the best fit
            fig = plt.figure()
            plt.plot(curr_wavs, curr_peak)
            plt.plot(curr_wavs, y_values , 'r-')
            # add the metrics to the plot
            plt.figtext(0.15, 0.85, 'R^2: {:.4f}'.format(r_squared))
            plt.figtext(0.15, 0.80, 'SSE: {:.4f}'.format(sse))
            
            # Clear some variables to be safe
            mus = []
            amps = []
            sigmas = []
            asymms = []
            curr_df = pd.DataFrame()
            
            # Calculate the peak dataframe
            mus = params[np.arange(0,
                             params.shape[0],
                             4)]
            amps = params[np.arange(1,
                                  params.shape[0],
                                  4)]
            sigmas = params[np.arange(2,
                                  params.shape[0],
                                  4)]
            asymms = params[np.arange(3,
                                  params.shape[0],
                                  4)]
            
            curr_df = pd.DataFrame([mus, amps, sigmas, asymms]).T
            curr_df.columns = ['Amplitude', 'Center', 'Width', 'Asymmetry']
            
            for _, row in curr_df.iterrows():
                plt.vlines(x=row['Center'],
                            ymin=0,
                           ymax=row['Amplitude'],
                           colors='g',
                           linestyles='dotted')
                plt.hlines(y=row['Amplitude'] * 0.6,
                           xmin=row['Center'] - row['Width']/2,
                           xmax=row['Center'] + row['Width']/2,
                           colors='g',
                           linestyles='dotted')
            
            # Save the current plot output
            current_output_time = datetime.now().timestamp()
            this_plot = f"{curr_x}_{curr_y}_{current_output_time}.png"
            fig.savefig(os.path.join("fitting_succeeded", this_plot))
            
            # Archive the current data in a dict
            peak_data[f"{curr_x}_{curr_y}"] = curr_df
        else:
            # Pickle the crucial data so we could rerun analysis on the results
            # open a file
            this_fname = f"{curr_x}_{curr_y}_{current_output_time}.pickle"
            this_fname = os.path.join("fitting_failed", this_fname)
            file = open(this_fname, 'wb')
            # Build the output dict
            data_out = {}
            data_out['curr_x'] = curr_x
            data_out['curr_y'] = curr_y
            data_out['curr_start'] = curr_start
            data_out['curr_end'] = curr_end
            data_out['curr_spectrum'] = curr_spectrum
            data_out['wav_array'] = wav_array
            
            pickle.dump(data_out, file)
            file.close()

# Pickle the crucial data so we could rerun analysis on the results
# open a file
this_fname = f"fitted_peaks_{current_output_time}.pickle"
this_fname = os.path.join("fitting_succeeded", this_fname)
file = open(this_fname, 'wb')
# Build the output dict
pickle.dump(peak_data, file)
file.close()

  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide

...

/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))
/tmp/ipykernel_94179/1733667475.py:5: RuntimeWarning: divide by zero encountered in divide
  return si * np.exp(-0.5 * ((lmbda - mu_i) ** 2 / (sigma_i - k_i * (lmbda - mu_i)) ** 2))


In [123]:
# truncate_range.shape


20